In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
# Import auxiliary libraries
import numpy as np

# Import Qiskit
from qiskit import IBMQ, Aer
from qiskit.algorithms import QAOA, VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.providers.aer.noise.noise_model import NoiseModel

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

import qiskit.test.mock as Fake

In [3]:
def cropyield_quadratic_program():
    cropyield = QuadraticProgram(name="Crop Yield")
    cropyield.integer_var(name="Wheat",lowerbound=0,upperbound=1)
    cropyield.integer_var(name="Soybeans",lowerbound=0,upperbound=1)
    cropyield.integer_var(name="Maize",lowerbound=0,upperbound=1)
    cropyield.integer_var(name="PushPull",lowerbound=0,upperbound=1)
    cropyield.maximize(
        quadratic={("Wheat","Soybeans"):2.4,("Wheat","Maize"):4,
                   ("Wheat","PushPull"):4,("Soybeans","Maize"):2,
                   ("Soybeans","PushPull"):1,("Maize","PushPull"):5},
        linear={"Wheat":2,"Soybeans":1,"Maize":4,"PushPull":0},
    )
    cropyield.linear_constraint(linear={"Wheat":1,"Soybeans":1,"Maize":1,"PushPull":1}, sense="<=",rhs=3)
    print(cropyield.export_as_lp_string())
    return cropyield

In [4]:
from qc_grader import grade_ex1b

cropyield = cropyield_quadratic_program()
grade_ex1b(cropyield)

/opt/conda/lib/python3.8/site-packages/pyscf/lib/misc.py:47: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Crop Yield

Maximize
 obj: 2 Wheat + Soybeans + 4 Maize + [ 4.800000000000 Wheat*Soybeans
      + 8 Wheat*Maize + 8 Wheat*PushPull + 4 Soybeans*Maize
      + 2 Soybeans*PushPull + 10 Maize*PushPull ]/2
Subject To
 c0: Wheat + Soybeans + Maize + PushPull <= 3

Bounds
       Wheat <= 1
       Soybeans <= 1
       Maize <= 1
       PushPull <= 1

Generals
 Wheat Soybeans Maize PushPull
End

Submitting your answer for ex1/partB. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [5]:
# Definitely real Qiskit module names
qiskit_module_names = [
    "Qiskit Finance",
    "Qiskit Machine Learning",
    "Qiskit Optimization",
    "Qiskit Nature",
]

In [6]:
from qc_grader import grade_ex1a

grade_ex1a(qiskit_module_names)

Submitting your answer for ex1/partA. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [7]:
ising_operations, _ = (
    QuadraticProgramToQubo()
    .convert(
        cropyield,
    )
    .to_ising()
)
print(f"Number of qubits required is {ising_operations.num_qubits}")

Number of qubits required is 6


In [8]:
QuadraticProgramToQubo().convert(cropyield)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Crop Yield

Minimize
 obj: - 160.400000000000 Wheat@0 - 159.400000000000 Soybeans@0
      - 162.400000000000 Maize@0 - 158.400000000000 PushPull@0
      - 158.400000000000 c0@int_slack@0 - 316.800000000000 c0@int_slack@1 + [
      52.800000000000 Wheat@0^2 + 100.800000000000 Wheat@0*Soybeans@0
      + 97.600000000000 Wheat@0*Maize@0 + 97.600000000000 Wheat@0*PushPull@0
      + 105.600000000000 Wheat@0*c0@int_slack@0
      + 211.200000000000 Wheat@0*c0@int_slack@1 + 52.800000000000 Soybeans@0^2
      + 101.600000000000 Soybeans@0*Maize@0
      + 103.600000000000 Soybeans@0*PushPull@0
      + 105.600000000000 Soybeans@0*c0@int_slack@0
      + 211.200000000000 Soybeans@0*c0@int_slack@1 + 52.800000000000 Maize@0^2
      + 95.600000000000 Maize@0*PushPull@0
      + 105.600000000000 Maize@0*c0@int_slack@0
      + 211.200000000000 Maize@0*c0@int_slack@1 + 52.800000000000 PushPull@0^2
      + 105.600000000000 PushPu

In [9]:
# We will use the Aer provided QASM simulator
backend = Aer.get_backend("qasm_simulator")

# Given we are using a simulator, we will fix the algorithm seed to ensure our results are reproducible
algorithm_globals.random_seed = 271828

In [10]:
def get_classical_solution_for(quadprog: QuadraticProgram):
    # Create solver
    solver = NumPyMinimumEigensolver()
    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)
    # Return result from optimizer
    return optimizer.solve(quadprog)

In [11]:
# Get classical result
classical_result = get_classical_solution_for(cropyield)
# Format and print result
print("Solution found using the classical method:\n")
print(f"Maximum crop-yield is {classical_result.fval} tons")
print(f"Crops used are: ")
_crops = [v.name for v in cropyield.variables]
for cropIndex, cropHectares in enumerate(classical_result.x):
    print(f"\t{cropHectares} ha of {_crops[cropIndex]}")


Solution found using the classical method:

Maximum crop-yield is 19.0 tons
Crops used are: 
	1.0 ha of Wheat
	0.0 ha of Soybeans
	1.0 ha of Maize
	1.0 ha of PushPull


In [12]:
def get_QAOA_solution_for(
    quadprog: QuadraticProgram, quantumInstance: QuantumInstance, optimizer=None,
):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create solver
    solver = QAOA(
        optimizer=optimizer, quantum_instance=quantumInstance, callback=callback,
    )

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

In [13]:
# Create a QuantumInstance
simulator_instance = QuantumInstance(
    backend=backend,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

# Get QAOA result
qaoa_result, qaoa_eval_count = get_QAOA_solution_for(cropyield, simulator_instance)

# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum crop-yield is {qaoa_result.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

Solution found using the QAOA method:

Maximum crop-yield is 19.0 tons
Crops used are: 
	1.0 ha of Wheat
	0.0 ha of Soybeans
	1.0 ha of Maize
	1.0 ha of PushPull

The solution was found within 3 evaluations of QAOA.


In [14]:
def get_VQE_solution_for(
    quadprog: QuadraticProgram, quantumInstance: QuantumInstance, optimizer=None,
):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create solver and optimizer
    solver = VQE(
        optimizer=optimizer, quantum_instance=quantumInstance, callback=callback
    )

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

In [15]:
# Create a QuantumInstance
simulator_instance = QuantumInstance(
    backend=backend,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

# Get VQE result
vqe_result, vqe_eval_count = get_VQE_solution_for(cropyield, simulator_instance)

# Format and print result
print("Solution found using the VQE method:\n")
print(f"Maximum crop-yield is {vqe_result.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(vqe_result.x, vqe_result.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {vqe_eval_count} evaluations of VQE")

Solution found using the VQE method:

Maximum crop-yield is 19.0 tons
Crops used are: 
	1.0 ha of Wheat
	0.0 ha of Soybeans
	1.0 ha of Maize
	1.0 ha of PushPull

The solution was found within 25 evaluations of VQE


In [16]:
from qc_grader import grade_ex1c

max_yield_qaoa = qaoa_result.fval
max_yield_vqe = vqe_result.fval

grade_ex1c(tonnage_qaoa=max_yield_qaoa, tonnage_vqe=max_yield_vqe)

Submitting your answer for ex1/partC. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [17]:
fake_device = Fake.FakeJohannesburg()

In [18]:
import qiskit.tools.jupyter

fake_device

<FakeJohannesburg('fake_johannesburg')>

In [19]:
# Create the noise model, which contains the basis gate set
noise_model = NoiseModel.from_backend(fake_device)

# Get the coupling map
coupling_map = fake_device.configuration().coupling_map

In [20]:
fake_instance = QuantumInstance(
    backend=backend,
    basis_gates=noise_model.basis_gates,
    coupling_map=coupling_map,
    noise_model=noise_model,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

In [21]:
# Get QAOA result
qaoa_result, qaoa_eval_count = get_QAOA_solution_for(cropyield, fake_instance)

# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum crop-yield is {qaoa_result.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")# Get QAOA result
qaoa_result, qaoa_eval_count = get_QAOA_solution_for(cropyield, fake_instance)

# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum crop-yield is {qaoa_result.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

Solution found using the QAOA method:

Maximum crop-yield is 19.0 tons
Crops used are: 
	1.0 ha of Wheat
	0.0 ha of Soybeans
	1.0 ha of Maize
	1.0 ha of PushPull

The solution was found within 3 evaluations of QAOA.
Solution found using the QAOA method:

Maximum crop-yield is 19.0 tons
Crops used are: 
	1.0 ha of Wheat
	0.0 ha of Soybeans
	1.0 ha of Maize
	1.0 ha of PushPull

The solution was found within 3 evaluations of QAOA.


In [22]:
# Function to estimate the number of qubits required
def estimate_number_of_qubits_required_for(max_hectares_per_crop, hectares_available):
    return int(
        4 * np.ceil(np.log2(max_hectares_per_crop + 1))
        + np.ceil(np.log2(hectares_available + 1))
    )


# Our new problem parameters
hectares_available = 10
max_hectares_per_crop = 10

# Retrieving the number of qubits required
number_of_qubits_required = estimate_number_of_qubits_required_for(
    max_hectares_per_crop=max_hectares_per_crop, hectares_available=hectares_available
)

print(
    f"Optimizing a {hectares_available} ha farm with each crop taking up to {max_hectares_per_crop} ha each,",
    f"the computation is estimated to require {number_of_qubits_required} qubits.",
)

Optimizing a 10 ha farm with each crop taking up to 10 ha each, the computation is estimated to require 20 qubits.


In [23]:
IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-09-13 13:52:50,743: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [24]:
provider = None
for prov in IBMQ.providers():
    if (
        "iqc-africa-21" in prov.credentials.hub
        and "q-challenge" in prov.credentials.group
        and "ex1" in prov.credentials.project
    ):
        # Correct provider found
        provider = prov
        
if provider == None:
    print("ERROR: The expected provider was not found!")
else:
    print("Yay! The expected provider was found!")

Yay! The expected provider was found!


In [25]:
backend_real = provider.get_backend("ibm_perth")

In [26]:
for _backend in IBMQ.get_provider(hub='ibm-q', group='open', project='main').backends():
    print(_backend.name())

ibmq_qasm_simulator
ibmq_armonk
ibmq_santiago
ibmq_bogota
ibmq_lima
ibmq_belem
ibmq_quito
simulator_statevector
simulator_mps
simulator_extended_stabilizer
simulator_stabilizer
ibmq_manila


In [27]:
%qiskit_backend_overview

In [28]:
backend_real

<IBMQBackend('ibm_perth') from IBMQ(hub='iqc-africa-21-1', group='q-challenge', project='ex1-rec2cu4ebhsj23XCY')>

In [29]:
quantum_instance_real = QuantumInstance(backend_real, shots=2048)

In [30]:
# Create a small crop-yield example quadratic program
cropyield_small = QuadraticProgram(name="Small Crop-Yield")

# Add two variables, indicating whether we grow 0, 1, or 2 hectares for two different crops
cropyield_small.integer_var(lowerbound=0, upperbound=2, name="Wheat")
cropyield_small.integer_var(lowerbound=0, upperbound=2, name="Maize")

# Add the objective function defining the yield in tonnes
cropyield_small.maximize(
    linear={"Wheat": 3, "Maize": 3},
    quadratic={("Maize", "Wheat"): 1, ("Maize", "Maize"): -2, ("Wheat", "Wheat"): -2},
)

# This linear constraint is not used as the model never reaches this. This is because the
# sum of the upperbounds on both variables is 4 already. If this constraint is applied, the
# model would require 6 qubits instead of 4.
# cropyield_small.linear_constraint(linear={"Wheat": 1, "Maize": 1}, sense="<=", rhs=4)

print(cropyield_small)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Small Crop-Yield

Maximize
 obj: 3 Wheat + 3 Maize + [ - 4 Wheat^2 + 2 Wheat*Maize - 4 Maize^2 ]/2
Subject To

Bounds
       Wheat <= 2
       Maize <= 2

Generals
 Wheat Maize
End



In [31]:
# Create a small crop-yield example quadratic program
cropyield_small = QuadraticProgram(name="Small Crop-Yield")

# Add two variables, indicating whether we grow 0, 1, or 2 hectares for two different crops
cropyield_small.integer_var(lowerbound=0, upperbound=2, name="Wheat")
cropyield_small.integer_var(lowerbound=0, upperbound=2, name="Maize")

# Add the objective function defining the yield in tonnes
cropyield_small.maximize(
    linear={"Wheat": 3, "Maize": 3},
    quadratic={("Maize", "Wheat"): 1, ("Maize", "Maize"): -2, ("Wheat", "Wheat"): -2},
)

# This linear constraint is not used as the model never reaches this. This is because the
# sum of the upperbounds on both variables is 4 already. If this constraint is applied, the
# model would require 6 qubits instead of 4.
# cropyield_small.linear_constraint(linear={"Wheat": 1, "Maize": 1}, sense="<=", rhs=4)

print(cropyield_small)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Small Crop-Yield

Maximize
 obj: 3 Wheat + 3 Maize + [ - 4 Wheat^2 + 2 Wheat*Maize - 4 Maize^2 ]/2
Subject To

Bounds
       Wheat <= 2
       Maize <= 2

Generals
 Wheat Maize
End



In [32]:
# Create our optimizer
optimizer = COBYLA(maxiter=1)

## Get result from real device with VQE
vqe_result_real, vqe_eval_count_real = get_VQE_solution_for(
    cropyield_small, quantum_instance_real, optimizer=optimizer
)

/opt/conda/lib/python3.8/site-packages/qiskit/utils/run_circuits.py:695: UserWarning: max_credits is not a recognized runtime option and may be ignored by the backend.
  return backend.run(circuits, **run_kwargs)


In [33]:
# Retrieve the VQE job sent
job_real = backend_real.jobs()[0]

print(f"VQE job created at {job_real.creation_date()} and has a job id of {job_real.job_id()}")

VQE job created at 2021-09-13 13:53:49.534000+00:00 and has a job id of 613f57ed6d2d52e3c3533f3f


In [36]:
from qc_grader import grade_ex1d

job_id = '613f57ed6d2d52e3c3533f3f'

grade_ex1d(job_id)

Submitting your answer for ex1/partD. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [35]:
print(provider)

<AccountProvider for IBMQ(hub='iqc-africa-21-1', group='q-challenge', project='ex1-rec2cu4ebhsj23XCY')>
